In [ ]:
from graphviz import Digraph
import logging

from pathlib import Path

import sys
import os
import json
import glob
from subprocess import STDOUT, PIPE, Popen, run, CalledProcessError

import json
    
logging.basicConfig(stream=sys.stdout)

logger = logging.getLogger('test.SystemTest')


def _prettify_url(url):
    words = url.split("/")
    parts = 0
    result=""

    for i in words[1:]:
        result = f"{result}/{i}"
        parts += 1
        if parts == 3:
            result = f"{result}\\n"
            parts = 0
    return result

def _prettify_params(params):
    if params is None:
        return ""

    logger.debug('_prettify_params')
    logger.debug(repr(params))
    
    pparams = json.dumps(params, default=str, ensure_ascii=False, indent=4)   
    logger.debug(repr(pparams))
    pparams = pparams.translate({
            ord("{"): "", 
            ord("}"): "" 
    })

    logger.debug(repr(pparams)) 
    pparams=pparams.rstrip()

    logger.debug(repr(pparams))
    pparams = "\n".join([ w[4:] for w in pparams.split("\n") ])

    logger.debug(repr(pparams))
    pparams = pparams.translate( {
            ord("\n"): "\\l",
            ord(" "): "\ " 
    })
    logger.debug(repr(pparams))

    return pparams+"\\l"

def run_pipeline(pipeline_name, step_list):
    #run sinara steps
    for step in step_list:
        step_dir = f'{pipeline_name}-{step}'
    
        with Popen(f"python step.dev.py", 
                                 shell=True, stdout=PIPE, stderr=STDOUT, cwd=step_dir) as child_process, open(f'{step_dir}.log', 'w') as logfile:

            for line in child_process.stdout:
                decoded_line = line.decode("utf-8")
                sys.stdout.write(decoded_line)
                logfile.write(decoded_line)
            child_process.communicate()
            #print(child_process.returncode )
            
            #if child_process.returncode != 0:
            #    raise Exception(f"SINARA Python module '{self.input_nb_name}' is failed!")


def show(graph_name, pipeline_name, step_list):

    t = Digraph(graph_name)
    t.attr(rank='max')
    t.attr('node', shape='record', fontsize='10', fontname = "Courier", color='orange', fillcolor='lightblue2', style='filled')
    t.attr('edge', shape='circle', color='orange1')

       
    #show sinara_notebooks
    for step_name in step_list:
        step_dir = f'{pipeline_name}-{step_name}/tmp/{step_name}'
        
        substeps = glob.glob(f"{step_dir}/*.json")
        for substep_report in substeps:
                
            with open(f'{substep_report}') as json_file:
               susbtep_content = json.load(json_file)

            #for k,v in self._dsml_notebooks.items(): 
            #    t.node(k,"{" + k + "|" + _prettify_params(v.params) + "}")
            #    t.attr('node', shape='Mrecord')
            #k = step_name + '-' + os.path.basename(substep)
            substep_name = Path(Path(substep_report).stem).stem
            #v = {}
            #v["params"] = {}
            t.node(step_name,"{" + step_name + "|" + substep_name + "}")
            t.attr('node', shape='Mrecord')
            
            #show resource catalogue

            #for k,v in self.entity_catalogue.items():
            #    t.node(v,"{" + k + "|" + _prettify_url(v) + "}")

            t.attr('node', shape='Mrecord', fontsize='10', fontname = "Courier", style='solid')

            #show dsml notebooks resources and artifacts

            #k = step + '-' + os.path.basename(substep) 
            a7s = susbtep_content.get("outputs")
            r7s = susbtep_content.get("inputs")

            if r7s is not None:
                for rk, rv in r7s.items():
                    #t.node(rv,"{" + rk + "|" + _prettify_url(rv) + "}")
                    #t.edge(rv,k)
                    
                    t.node(rk,"{" + rk + "}")
                    t.edge(rk,step_name)
                    


            if a7s is not None:
                for ak, av in a7s.items():
                    #t.node(av,"{" + ak + "|" + _prettify_url(av) + "}")
                    #t.edge(k,av)
                    
                    t.node(ak,"{" + ak + "}")
                    t.edge(step_name,ak)

    return t

pipeline_name = 'pipeline'
step_list = ['step1', 'step2', 'step3', 'step4']
graph_name = 'test'

os.environ["DESIGN_MODE"] = 'TRUE'
run_pipeline(pipeline_name, step_list)
show(graph_name, pipeline_name, step_list)